In [204]:
import json

file_path = '../stage1_problems/STAGE1_2.json'

with open(file_path, 'r') as file:
    data = json.load(file)



In [205]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [206]:
class Order:
    def __init__(self, order_info):
        # [ORD_ID, ORD_TIME, SHOP_LAT, SHOP_LON, DLV_LAT, DLV_LON, VOL, COOK_TIME, DLV_DEADLINE]
        self.id = order_info[0]
        self.order_time = order_info[1]
        self.shop_lat = order_info[2]
        self.shop_lon = order_info[3]
        self.dlv_lat = order_info[4]
        self.dlv_lon = order_info[5]
        self.cook_time = order_info[6]
        self.volume = order_info[7]
        self.deadline = order_info[8]

        self.ready_time = self.order_time + self.cook_time

    def __repr__(self) -> str:
        return f'Order([{self.id}, {self.order_time}, {self.shop_lat}, {self.shop_lon}, {self.dlv_lat}, {self.dlv_lon}, {self.volume}, {self.cook_time}, {self.deadline}])'

# 배달원 class
class Rider:
    def __init__(self, rider_info):
        # [type, speed, capa, var_cost, fixed_cost, service_time, available number]
        self.type = rider_info[0]
        self.speed = rider_info[1]
        self.capa = rider_info[2]
        self.var_cost = rider_info[3]
        self.fixed_cost = rider_info[4]
        self.service_time = rider_info[5]
        self.available_number = rider_info[6]
    
    def __repr__(self) -> str:
        return f'Rider([{self.type}, {self.speed}, {self.capa}, {self.var_cost}, {self.fixed_cost}, {self.service_time}, {self.available_number}])'
    
    # 주어진 거리에 대한 배달원 비용 계산
    # = 배달원별 고정비 + 이동거리로 계산된 변동비
    def calculate_cost(self, dist):
        return self.fixed_cost + dist / 100.0 * self.var_cost

# 묶음 주문 정보
class Bundle:
    def __init__(self, all_orders, rider, shop_seq, dlv_seq, total_volume, total_dist, feasible=True):
        self.rider = rider
        self.all_orders = all_orders
        self.feasible = feasible
        self.shop_seq = shop_seq
        self.dlv_seq = dlv_seq
        self.total_volume = total_volume
        self.total_dist = total_dist
        self.update_cost()

    # 묶음 주문의 비용 update
    def update_cost(self):
        self.cost = self.rider.calculate_cost(self.total_dist)
        self.cost_per_ord = self.cost / len(self.shop_seq)


    def __repr__(self) -> str:
        return f'Bundle(all_orders, {self.rider.type}, {self.shop_seq}, {self.dlv_seq}, {self.total_volume}, {self.feasible})'


In [207]:
riders_data = data['RIDERS']
orders_data = data['ORDERS']
dist_data = data['DIST']

riders_columns = ['Transport', 'Speed', 'Capacity', 'RestTime', 'Cost', 'Income', 'Satisfaction']
riders_df = pd.DataFrame(riders_data, columns=riders_columns)

orders_columns = ['OrderID', 'Time', 'StartLat', 'StartLong', 'EndLat', 'EndLong', 'Distance', 'TimeWindow', 'deadline']
orders_df = pd.DataFrame(orders_data, columns=orders_columns)

dist_df = pd.DataFrame(dist_data)
dist_array = dist_df.to_numpy()

In [208]:
orders_df

,OrderID,Time,StartLat,StartLong,EndLat,EndLong,Distance,TimeWindow,deadline
0,0,17,36.011127,126.013719,36.006495,126.011111,1200,24,3384
1,1,132,35.999656,126.006998,35.999958,126.016897,1500,48,3907
2,2,208,35.996656,126.003360,35.995848,126.005397,300,11,2556
3,3,286,36.025792,126.004831,36.024556,126.005798,900,34,3220
4,4,296,36.001350,125.974501,35.995763,125.974194,900,16,3382
...,...,...,...,...,...,...,...,...,...
95,95,3468,36.008820,125.953697,35.998096,125.979487,1500,24,7812
96,96,3479,35.995795,126.011387,36.002826,126.021588,600,18,6458
97,97,3553,36.001045,126.008804,35.994064,126.032792,1800,34,8092
98,98,3567,36.001045,126.019247,35.984750,126.007104,600,25,6847


In [209]:
riders_df

,Transport,Speed,Capacity,RestTime,Cost,Income,Satisfaction
0,BIKE,5.291005,100,60,8000,120,20
1,WALK,1.322751,70,30,8000,120,30
2,CAR,4.232804,200,100,6000,180,100


In [210]:
K = data['K']

all_orders = [Order(order_info) for order_info in data['ORDERS']]
all_riders = [Rider(rider_info) for rider_info in data['RIDERS']]

dist = np.array(data['DIST'])
for r in all_riders:
    r.T = np.round(dist/r.speed + r.service_time)

In [211]:
# 주문들의 총 부피 계산
# shop_seq는 주문들의 pickup list
# Note: shop_seq는 주문 id의 list와 동일
def get_total_volume(all_orders, shop_seq):
    return sum(all_orders[k].volume for k in shop_seq)

# shop_seq의 순서로 pickup하고 dlv_seq 순서로 배달할 때 총 거리 계산
# Note: shop_seq 와 dlv_seq는 같은 주문 id들을 가져야 함. 즉, set(shop_seq) == seq(dlv_seq). (주문 id들의 순서는 바뀔 수 있음)
def get_total_distance(K, dist_mat, shop_seq, dlv_seq):
    return sum(dist_mat[i,j] for (i,j) in zip(shop_seq[:-1], shop_seq[1:])) + dist_mat[shop_seq[-1], dlv_seq[0]+K] + sum(dist_mat[i+K,j+K] for (i,j) in zip(dlv_seq[:-1], dlv_seq[1:]))

# shop_seq의 순서로 pickup하고 dlv_seq 순서로 배달할 때 pickup과 delivery시간을 반환
# Note: shop_seq 와 dlv_seq는 같은 주문 id들을 가져야 함. 즉, set(shop_seq) == seq(dlv_seq). (주문 id들의 순서는 바뀔 수 있음)
def get_pd_times(all_orders, rider, shop_seq, dlv_seq):
    
    K = len(all_orders)

    pickup_times = {}

    k = shop_seq[0]
    t = all_orders[k].order_time + all_orders[k].cook_time # order time + order cook time
    pickup_times[k] = t
    for next_k in shop_seq[1:]:
        t = max(t+rider.T[k, next_k], all_orders[next_k].ready_time) # max{travel time + service time, ready time}
        pickup_times[next_k] = t
                
        k = next_k

    dlv_times = {}

    k = dlv_seq[0]
    t += rider.T[shop_seq[-1], k + K]

    dlv_times[k] = t
        
    for next_k in dlv_seq[1:]:
        t += rider.T[k + K, next_k + K]

        dlv_times[next_k] = t

        k = next_k

    return pickup_times, dlv_times

In [212]:
r_car = np.round(dist/all_riders[2].speed + all_riders[2].service_time)
r_bike = np.round(dist/all_riders[0].speed + all_riders[0].service_time)
r_walk = np.round(dist/all_riders[1].speed + all_riders[1].service_time)

#### get pd times를 사용하면 위의 r_car, r_bike, r_walk 사용할 필요 없음 실행 시간 확인해서 결정
#### get total distance를 사용해서 순서에 맞는 거리 계산
#### get total volume를 사용해서 총 부피 계산

In [213]:
import time
import random

In [214]:
all_riders

[Rider([BIKE, 5.291005291005291, 100, 60, 8000, 120, 20]),
 Rider([WALK, 1.3227513227513228, 70, 30, 8000, 120, 30]),
 Rider([CAR, 4.2328042328042335, 200, 100, 6000, 180, 100])]

In [291]:
def generate_random_solution(all_orders, all_riders, dist_matrix):
    solution_dict = {}
    orders = list(range(len(all_orders)))
    random.shuffle(orders)
    
    rider_counts = {r.type: r.available_number for r in all_riders} # 라이더 타입별 수
    rider_ids = {r.type: list(range(r.available_number)) for r in all_riders} # 라이더 타입별 id 리스트
    # 그냥 dict 말고 list 두개 만들어서 해도 될듯?

    while orders:
        for rider in all_riders:
            if not orders or rider_counts[rider.type] == 0:
                continue
            
            # 랜덤하게 묶음 크기를 결정 (최소 1, 최대 5)
            bundle_size = random.randint(1, min(len(orders), 2))
            selected_orders = orders[:bundle_size]
            
            total_volume = get_total_volume(all_orders, selected_orders)
            if total_volume > rider.capa:
                continue

            total_dist = get_total_distance(len(all_orders), dist_matrix, selected_orders, selected_orders)
            pickup_times, dlv_times = get_pd_times(all_orders, rider, selected_orders, selected_orders)
            if any(dlv_times[k] > all_orders[k].deadline for k in selected_orders):
                continue

            orders = orders[bundle_size:]
            
            for order in selected_orders:
                if rider_ids[rider.type]:
                    rider_id = rider_ids[rider.type].pop()
                    solution_dict[order] = (rider.type, rider_id)
            
            rider_counts[rider.type] -= 1

    return solution_dict

In [292]:
import sys

In [307]:
def fitness_function(solution_dict, all_orders, all_riders, dist_matrix):
    rider_bundles = {(r.type, i): [] for r in all_riders for i in range(r.available_number)}
    for order_idx, (rider_type, rider_id) in solution_dict.items(): # order_idx: 주문 번호, rider_type: 라이더 타입, rider_id: 라이더 번호
        rider_bundles[(rider_type, rider_id)].append(order_idx)

    total_cost = 0
    for (rider_type, rider_id), order_indices in rider_bundles.items():
        if not order_indices:
            continue
        rider = next(r for r in all_riders if r.type == rider_type)
        
        # 주문들에 대한 총 부피 계산
        total_volume = get_total_volume(all_orders, order_indices)
        if total_volume > rider.capa:
            return sys.maxsize

        shop_seq = order_indices
        dlv_seq = order_indices
        dist = get_total_distance(len(all_orders), dist_matrix, shop_seq, dlv_seq)
        cost = rider.calculate_cost(dist)
        total_cost += cost

        # 제약조건 확인
        _, dlv_times = get_pd_times(all_orders, rider, shop_seq, dlv_seq)
        for k, dlv_time in dlv_times.items():
            if dlv_time > all_orders[k].deadline:
                return sys.maxsize

    return total_cost

In [308]:
from scipy.optimize import minimize

In [309]:
initial_solution = generate_random_solution(all_orders, all_riders, dist_array)

In [310]:
initial_x = []
for order, (rider_type, rider_id) in sorted(initial_solution.items()):
    rider_type_index = next(i for i, r in enumerate(all_riders) if r.type == rider_type)
    initial_x.append(rider_type_index)
    initial_x.append(rider_id)

In [311]:
def optimization_function(x):
    solution_dict = {}
    for i in range(0, len(x), 2):
        rider_type_index = int(x[i])
        rider_id = int(x[i + 1])
        order_id = i // 2
        rider_type = all_riders[rider_type_index % len(all_riders)].type
        solution_dict[order_id] = (rider_type, rider_id)
    return fitness_function(solution_dict, all_orders, all_riders, dist_array)


In [312]:
result = minimize(optimization_function, initial_x, method='trust-constr', 
                  options={'verbose': 1, 'maxiter': 1000, 'gtol': 1e-6, 'xtol': 1e-6})

`gtol` termination condition is satisfied.
Number of iterations: 1, function evaluations: 195, CG iterations: 0, optimality: 0.00e+00, constraint violation: 0.00e+00, execution time: 0.0015 s.


In [313]:
result.fun

9223372036854775807

In [314]:
optimized_solution = {}
for i in range(0, len(result.x), 2):
    rider_type_index = int(result.x[i])
    rider_id = int(result.x[i + 1])
    order_id = i // 2
    rider_type = all_riders[rider_type_index % len(all_riders)].type
    optimized_solution[order_id] = (rider_type, rider_id)

optimized_fitness_value = fitness_function(optimized_solution, all_orders, all_riders, dist_array)


In [315]:
optimized_fitness_value

9223372036854775807